In [222]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/restaurant-reviews/Restaurant_Reviews.tsv


In [223]:
data=pd.read_csv('/kaggle/input/restaurant-reviews/Restaurant_Reviews.tsv',delimiter='\t',quoting=3)
data

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [224]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
stop=stopwords.words('english');
stop.remove('not')
cleaned_reviews=[]
for i in range(0,1000):
    #removing punctuations
    review=re.sub('[^a-zA-Z]',' ',data['Review'][i])
    review=review.lower()
    review=review.split()
    stemmer=PorterStemmer()
    review=[stemmer.stem(word) for word in review if word not in stop]
    review=' '.join(review)
    cleaned_reviews.append(review)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [225]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
vectorizer=CountVectorizer(max_features=1000)
#vectorizer = TfidfVectorizer(max_features=1000)
X=vectorizer.fit_transform(cleaned_reviews).toarray()
y=data.iloc[:,-1].values
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [226]:
from sklearn.model_selection import train_test_split
X_tr,X_ts,Y_tr,Y_ts=train_test_split(X,y,random_state=1)


In [227]:
# from sklearn.naive_bayes import BernoulliNB
# clf=BernoulliNB()

# from sklearn.tree import DecisionTreeClassifier
# clf=DecisionTreeClassifier()

# from sklearn.ensemble import RandomForestClassifier
# clf=RandomForestClassifier(n_estimators=500)

from sklearn.svm import SVC
clf=SVC(C=0.9,kernel='rbf')

# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression()

# from sklearn.neighbors import KNeighborsClassifier
# clf = KNeighborsClassifier(n_neighbors=3)


clf.fit(X_tr,Y_tr)
Y_pr=clf.predict(X_ts)
print(clf.score(X_ts,Y_ts))

0.824


In [228]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_ts,Y_pr))
print(confusion_matrix(Y_ts,Y_pr))

              precision    recall  f1-score   support

           0       0.80      0.90      0.85       134
           1       0.87      0.73      0.79       116

    accuracy                           0.82       250
   macro avg       0.83      0.82      0.82       250
weighted avg       0.83      0.82      0.82       250

[[121  13]
 [ 31  85]]


In [229]:
print("Enter Your Review :")
new_review = str(input())
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
new_review = [ps.stem(word) for word in new_review if not word in set(stop)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = vectorizer.transform(new_corpus).toarray()
new_y_pred = clf.predict(new_X_test)
if(new_y_pred[0]):
    print("Positive")
else:
    print("Negative")

Enter Your Review :


 good poor bad very nice great


Positive
